In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/msnhealthnews.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header)
frame.head()

,tweetid,date,title
0,220146274970763264,Tue Jul 03 13:25:36 +0000 2012,Heavy Coffee Intake May Affect Fertility Treat...
1,220146274236760065,Tue Jul 03 13:25:36 +0000 2012,"Summer Is Peak Time for Teens to Try Drugs, Al..."
2,220146273444040704,Tue Jul 03 13:25:36 +0000 2012,Some Crash-Avoidance Systems May Work Better T...
3,220146272638734336,Tue Jul 03 13:25:36 +0000 2012,More Genes Linked to Osteoarthritis Identified...
4,219891972108845056,Mon Jul 02 20:35:06 +0000 2012,No Health Risk When Jehovah's Witnesses Refuse...


In [3]:
# -------------------------------------------------
frame.shape

(3199, 3)

In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Heavy Coffee Intake May Affect Fertility Treatments: Study:  http://on-msn.com/P5E3ge'

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://on-msn.com/P5E3ge
http://on-msn.com/O01Nw2
http://on-msn.com/O01O3l
http://on-msn.com/O01NfM
http://on-msn.com/NXt7v8
http://on-msn.com/NXt7v4
http://on-msn.com/NXt8z1
http://on-msn.com/NXt8yZ
http://on-msn.com/P1D3tz
http://on-msn.com/NeSut8
http://on-msn.com/NeSut6
http://on-msn.com/NeSut2
http://on-msn.com/OdGOcK
http://on-msn.com/OZdyZL
http://on-msn.com/NVX26I
http://on-msn.com/OZdyJv
http://on-msn.com/OZdB81
http://on-msn.com/OZdyJr
http://on-msn.com/MjswZ7
http://on-msn.com/O2ep9v
http://on-msn.com/OL6zUn
http://on-msn.com/NLTow6
http://on-msn.com/MdmmcN
http://on-msn.com/MdmlWz
http://on-msn.com/MdmlWx
http://on-msn.com/QBsri0
http://on-msn.com/MdmlWv
http://on-msn.com/OJsXgW
http://on-msn.com/NcsxZ4
http://on-msn.com/NcsxYY
http://on-msn.com/OJsVFP
http://on-msn.com/NcsxYW
http://on-msn.com/NWKnns
http://on-msn.com/OEYkJe
http://on-msn.com/MZgroe
http://on-msn.com/NSBSK9
http://on-msn.com/OEa9j3
http://on-msn.com/OEa8eZ
http://on-msn.com/OEa8eV
http://on-msn.com/NGAgQb


http://on-msn.com/yhM4ig
http://on-msn.com/x1VCfm
http://on-msn.com/xsRpMK
http://on-msn.com/xSrloS
http://on-msn.com/AAdLYN
http://on-msn.com/xQFcM3
http://on-msn.com/xBldub
http://on-msn.com/AenMYc
http://on-msn.com/AdK350
http://on-msn.com/w52p1v
http://on-msn.com/wRYRYs
http://on-msn.com/xLKrTd
http://on-msn.com/zfoJfm
http://on-msn.com/wMUsPE
http://on-msn.com/zISTWO
http://on-msn.com/xdHUAC
http://on-msn.com/w3m3U1
http://on-msn.com/zPk8Qz
http://on-msn.com/zAgHwp
http://on-msn.com/wnLECm
http://on-msn.com/zZOiHV
http://on-msn.com/xSB7T3
http://on-msn.com/y7ufOU
http://on-msn.com/yovEgW
http://on-msn.com/z9kk8J
http://on-msn.com/AA7WOc
http://on-msn.com/wMq218
http://on-msn.com/y9fsN7
http://on-msn.com/ABFiw5
http://on-msn.com/zxhTpK
http://on-msn.com/wFOx6n
http://on-msn.com/ADKoQo
http://on-msn.com/yVu6nQ
http://on-msn.com/zzuUpG
http://on-msn.com/zqwy6T
http://on-msn.com/xmvp1c
http://on-msn.com/An5ALG
http://on-msn.com/wmSTS6
http://on-msn.com/yRuQc4
http://on-msn.com/AoZWsh


http://on-msn.com/pXrgkj
http://on-msn.com/qi2lgn
http://on-msn.com/mQWIZw
http://on-msn.com/p2Vn6A
http://on-msn.com/pCl3If
http://on-msn.com/oxYUVh
http://on-msn.com/mTonWM
http://on-msn.com/mTqX1g
http://on-msn.com/orw0Ql
http://on-msn.com/rrVZ6C
http://on-msn.com/peP7RA
http://on-msn.com/oPBYHT
http://on-msn.com/rcrHVL
http://on-msn.com/mZcY8n
http://on-msn.com/mP5q6i
http://on-msn.com/oQBiLs
http://on-msn.com/p4v74A
http://on-msn.com/pzcnLB
http://on-msn.com/pFNNaU
http://on-msn.com/qhW0U8
http://on-msn.com/ovJJBR
http://on-msn.com/oPofms
http://on-msn.com/oQgCzA
http://on-msn.com/qvuQQX
http://on-msn.com/oDU0ya
http://on-msn.com/pPjN1V
http://on-msn.com/pxaytc
http://on-msn.com/pJc07w
http://on-msn.com/qbr59r
http://on-msn.com/n3ikmC
http://on-msn.com/rgHoPy
http://on-msn.com/oNPxR3
http://on-msn.com/qnMHo4
http://on-msn.com/ppKpKh
http://on-msn.com/pzPj2Z
http://on-msn.com/rbXwTd
http://on-msn.com/qDH6Fk
http://on-msn.com/ovEqFn
http://on-msn.com/p0ua5D
http://on-msn.com/ngNPF0


http://on-msn.com/K1lylN
Two-Drug Combo Helps Teens With Migraines:  http://on-msn.com/K1lylF


http://on-msn.com/K1lylF
Fitness Fights High Blood Pressure, Even With Family History:  http://on-msn.com/KLIV58


http://on-msn.com/KLIV58
Doctors Restore Some Hand Function to Quadriplegic Patient:  http://on-msn.com/JCmFdJ


http://on-msn.com/JCmFdJ
Are Post-Angioplasty Stress Tests Unnecessary?:  http://on-msn.com/JCmFdH


http://on-msn.com/JCmFdH
Inhaled Steroids Lead to Big Drop in Asthma Deaths at Texas Hospital: Study:  http://on-msn.com/JCmCPf


http://on-msn.com/JCmCPf
FDA Panel to Consider At-Home HIV Test:  http://on-msn.com/JECjo0


http://on-msn.com/JECjo0
Steroid Nasal Sprays Show Small Benefit for Sinusitis: Study:  http://on-msn.com/JEChMZ


http://on-msn.com/JEChMZ
Some Sports May Help Protect Men's Bones:  http://on-msn.com/LK9vva


http://on-msn.com/LK9vva
Pot Might Help Ease Multiple Sclerosis Symptoms:  http://on-msn.com/JPXd28


http://on-msn.com/JPXd28
Acupuncture May


http://on-msn.com/HH4SS1
Fewer U.S. Med Students Choose Psychiatry: Report:  http://on-msn.com/HA95nc


http://on-msn.com/HA95nc
Study Weighs In on Diets for Kids:  http://on-msn.com/HA97vk


http://on-msn.com/HA97vk
Justices Meet to Decide Fate of Health-Reform Legislation:  http://on-msn.com/Hns9KN


http://on-msn.com/Hns9KN
When Paramedics Suspect Stroke, They're Usually Right:  http://on-msn.com/Ht4CF3


http://on-msn.com/Ht4CF3
Obstructive Sleep Apnea May Be Linked to Depression:  http://on-msn.com/Hnsayr


http://on-msn.com/Hnsayr
Two-Thirds of Kids With Autism Have Been Bullied: Study:  http://on-msn.com/Hnsayj


http://on-msn.com/Hnsayj
Thyroid Surgery Riskier for Older Patients: Study:  http://on-msn.com/HolxFR


http://on-msn.com/HolxFR
Warm Weather Triggers Early Allergy Season:  http://on-msn.com/HpdAFe


http://on-msn.com/HpdAFe
'Superinfected' Patients Give Clues to Fighting HIV:  http://on-msn.com/Hmnw2u


http://on-msn.com/Hmnw2u
People Stir Up Millions of Bacteria Whe



http://on-msn.com/zZQnT5
Sleeplessness Tied to Early Alzheimer's, Study Says:  http://on-msn.com/wP7Jis


http://on-msn.com/wP7Jis
Vitamin D May Not Boost Heart Health in Kidney Patients:  http://on-msn.com/wzGGQH


http://on-msn.com/wzGGQH
FDA Thinks Shortage of Cancer Drug for Kids Can Be Averted:  http://on-msn.com/yE4EJd


http://on-msn.com/yE4EJd
Back Problems May Cut Into Retirement Savings:  http://on-msn.com/A7TsaP


http://on-msn.com/A7TsaP
Teens With Autism Preoccupied With TV, Video Games: Study:  http://on-msn.com/wBiJK8


http://on-msn.com/wBiJK8
Switching to Water, Diet Drinks Linked to Modest Weight Loss:  http://on-msn.com/wD89nH


http://on-msn.com/wD89nH
Lost Love Often the Biggest Source of Regret, Study Shows:  http://on-msn.com/xEbXaA


http://on-msn.com/xEbXaA
Bullying a Strong Risk Factor for Suicide in Gay Youth: Study:  http://on-msn.com/wTqB0r


http://on-msn.com/wTqB0r
Don't Forget Your Kids on Valentine's Day:  http://on-msn.com/zC6xFS


http://on-msn.com/

http://on-msn.com/tqljW2
Can Chewing Gum Boost Exam Scores?:  http://on-msn.com/sQZsKU


http://on-msn.com/sQZsKU
Americans Encouraged to Get Flu Vaccine:  http://on-msn.com/urfcc3


http://on-msn.com/urfcc3
Psych Episode Near Childbirth May Presage Bipolar Disorder:  http://on-msn.com/rLrfYV


http://on-msn.com/rLrfYV
'Love Hormone' May Buffer Kids From Mom's Depression:  http://on-msn.com/vAJ8dz


http://on-msn.com/vAJ8dz
Burn-Casualty Soldiers at High Risk for Kidney Injury, Study Finds:  http://on-msn.com/vKGsEr


http://on-msn.com/vKGsEr
Behavior Problems May Surface in Preemies by Preschool:  http://on-msn.com/ssW8WI


http://on-msn.com/ssW8WI
Obese Patients May Benefit the Most From Surgery for Irregular Heartbeat:  http://on-msn.com/sPXImZ


http://on-msn.com/sPXImZ
Recent E. Coli Outbreak Traced to Lettuce From One Farm: CDC:  http://on-msn.com/w39ZxE


http://on-msn.com/w39ZxE
For Older Runners, the Message Is: Keep Those Legs Pumping:  http://on-msn.com/uIqOza


http://on-ms



http://on-msn.com/pL6rHY
Stress May Affect Preemies' Brains, Study Shows:  http://on-msn.com/q7SAOn


http://on-msn.com/q7SAOn
Hospital Readmission Rates on the Rise in Older Adults: Study:  http://on-msn.com/nUrNKg


http://on-msn.com/nUrNKg
Older Dads Can Pass on Gene Mutations That Lower Intellect:  http://on-msn.com/pouAbx


http://on-msn.com/pouAbx
Experts Assess What Works for Weight Loss:  http://on-msn.com/nL1DJz


http://on-msn.com/nL1DJz
Preemies May Be at Higher Risk of Epilepsy Later in Life:  http://on-msn.com/qwB8WD


http://on-msn.com/qwB8WD
Facebook Pages May Offer Clues to Underage Drinking:  http://on-msn.com/nxif80


http://on-msn.com/nxif80
HPV-Linked Oral Cancers on the Rise, Study Finds:  http://on-msn.com/r8k09u


http://on-msn.com/r8k09u
Mom's Healthy Diet Might Cut Birth Defect Risk:  http://on-msn.com/nQ6CVE


http://on-msn.com/nQ6CVE
Conviction Rates Similar for Suspects of Child, Adult Murder in Utah:  http://on-msn.com/q19adv


http://on-msn.com/q19adv
Ea

In [6]:
frame.head()

,tweetid,date,title,url
0,220146274970763264,Tue Jul 03 13:25:36 +0000 2012,Heavy Coffee Intake May Affect Fertility Treat...,http://on-msn.com/P5E3ge
1,220146274236760065,Tue Jul 03 13:25:36 +0000 2012,"Summer Is Peak Time for Teens to Try Drugs, Al...",http://on-msn.com/O01Nw2
2,220146273444040704,Tue Jul 03 13:25:36 +0000 2012,Some Crash-Avoidance Systems May Work Better T...,http://on-msn.com/O01O3l
3,220146272638734336,Tue Jul 03 13:25:36 +0000 2012,More Genes Linked to Osteoarthritis Identified:,http://on-msn.com/O01NfM
4,219891972108845056,Mon Jul 02 20:35:06 +0000 2012,No Health Risk When Jehovah's Witnesses Refuse...,http://on-msn.com/NXt7v8


In [7]:
# -------------------------------------------------
frame_urls = frame['url'].values

In [8]:
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//div[@class=" card-content "]/p/text()',
               '',
               '']

In [ ]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):

    try:
        # Start the connection
        res = requests.get(url)

        # Get return content as text
        text_html = res.text

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            
            # Initialize XPath
            xpath_selector = Selector(text=text_html)

#             print("Before Scrapting ")
            # Scrapt content with XPath
            list_scrapt = scrapt_w_xpath(list_scrapt)
            print(list_scrapt)

            count_xp_scrapt = len(list_scrapt)
         
            
            if title_attempt:
                print("Title: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_scrapt.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)
                
                title_attempt = False

            for k_, v_ in list_scrapt.items():
#                 print("v_:", v_)
                frame[k_].iloc[key] = v_
            
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))

    except:
        print(url)
        pass
    

http://on-msn.com/P5E3ge
http://on-msn.com/O01Nw2
http://on-msn.com/O01O3l
http://on-msn.com/O01NfM
http://on-msn.com/NXt7v8
http://on-msn.com/NXt7v4
http://on-msn.com/NXt8z1
http://on-msn.com/NXt8yZ
http://on-msn.com/P1D3tz
http://on-msn.com/NeSut8
http://on-msn.com/NeSut6
http://on-msn.com/NeSut2
http://on-msn.com/OdGOcK
http://on-msn.com/OZdyZL
http://on-msn.com/NVX26I
http://on-msn.com/OZdyJv
http://on-msn.com/OZdB81
http://on-msn.com/OZdyJr
http://on-msn.com/MjswZ7
http://on-msn.com/O2ep9v
http://on-msn.com/OL6zUn
http://on-msn.com/NLTow6
http://on-msn.com/MdmmcN
http://on-msn.com/MdmlWz
http://on-msn.com/MdmlWx
http://on-msn.com/QBsri0
http://on-msn.com/MdmlWv
http://on-msn.com/OJsXgW
http://on-msn.com/NcsxZ4
http://on-msn.com/NcsxYY
http://on-msn.com/OJsVFP
http://on-msn.com/NcsxYW
http://on-msn.com/NWKnns
http://on-msn.com/OEYkJe
http://on-msn.com/MZgroe
http://on-msn.com/NSBSK9
http://on-msn.com/OEa9j3
http://on-msn.com/OEa8eZ
http://on-msn.com/OEa8eV
http://on-msn.com/NGAgQb


http://on-msn.com/MCyCjT
http://on-msn.com/MCyEs1
http://on-msn.com/KL7MEQ
http://on-msn.com/LhIgta
http://on-msn.com/Kk3kgW
http://on-msn.com/KHXaq8
http://on-msn.com/KHOHDl
http://on-msn.com/KHOIag
http://on-msn.com/KHqi0F
http://on-msn.com/KHqjlc
http://on-msn.com/KHqi0C
http://on-msn.com/MuGHHk
http://on-msn.com/JMiuyk
http://on-msn.com/MuGK69
http://on-msn.com/JMitdN
http://on-msn.com/JMiuye
http://on-msn.com/JLzJ2I
http://on-msn.com/JLzIfa
http://on-msn.com/Mtm8uQ
http://on-msn.com/KDI3hx
http://on-msn.com/L9It1E
http://on-msn.com/KqawsZ
http://on-msn.com/KqawsT
http://on-msn.com/L9dXok
http://on-msn.com/Kpx5xW
http://on-msn.com/KgKR5c
http://on-msn.com/KgKOWW
http://on-msn.com/KOQFBq
http://on-msn.com/Lp716g
http://on-msn.com/KOQDcE
http://on-msn.com/KBxWtn
http://on-msn.com/L7hLGM
http://on-msn.com/L7hNP1
http://on-msn.com/L7hLGE
http://on-msn.com/LlnckN
http://on-msn.com/L3Cziu
http://on-msn.com/L3CxHa
http://on-msn.com/L3Cz23
http://on-msn.com/L3Cz21
http://on-msn.com/KkGu9O


http://on-msn.com/IiutN3
http://on-msn.com/IAR91N
http://on-msn.com/IieEpD
http://on-msn.com/IFbTRt
http://on-msn.com/IFbWgb
http://on-msn.com/IFbWg8
http://on-msn.com/ILDOmC
http://on-msn.com/IOePdC
http://on-msn.com/IOeMOW
http://on-msn.com/IOePu2
http://on-msn.com/IRE9SW
http://on-msn.com/IDAd6g
http://on-msn.com/IDAd6e
http://on-msn.com/ItlnDQ
http://on-msn.com/IcaFuX
http://on-msn.com/IFW14U
http://on-msn.com/IzDvYc
http://on-msn.com/IzDvY9
http://on-msn.com/IzjZ0h
http://on-msn.com/IzjZ0e
http://on-msn.com/Iq3XXq
http://on-msn.com/Iq3XXn
http://on-msn.com/ILIirm
http://on-msn.com/IHYOWA
http://on-msn.com/IHYOWw
http://on-msn.com/IHYOWs
http://on-msn.com/IE9LNs
http://on-msn.com/IxzOSM
http://on-msn.com/IxzOSG
http://on-msn.com/IE9Lx6
http://on-msn.com/IE891D
http://on-msn.com/Iu6yfL
http://on-msn.com/Iu6yfH
http://on-msn.com/Iu6wV4
http://on-msn.com/IHsmGC
http://on-msn.com/IDB0mG
http://on-msn.com/Iulxst
http://on-msn.com/IRrPi2
http://on-msn.com/IRrPhS
http://on-msn.com/IRrOul


http://on-msn.com/zAhCIv
http://on-msn.com/yBInQu
http://on-msn.com/w5CcaF
http://on-msn.com/AkbS8Q
http://on-msn.com/wz02go
http://on-msn.com/w0KBhe
http://on-msn.com/ynViq0
http://on-msn.com/ztGbGU
http://on-msn.com/xViIiN
http://on-msn.com/w1l17i
http://on-msn.com/xV69WV
http://on-msn.com/ystKlj
http://on-msn.com/AiWTUt
http://on-msn.com/xvJbzU
http://on-msn.com/A431jW
http://on-msn.com/yYdJcf
http://tinyurl.com/7zult8y
http://on-msn.com/xnLYab
http://on-msn.com/zwoNGi
http://on-msn.com/xGsLcI
http://on-msn.com/zRhqvz
http://on-msn.com/wL9zXf
http://on-msn.com/AideiM
http://on-msn.com/AxjOJ7
http://on-msn.com/wBIpQD
http://on-msn.com/xMPskR
http://on-msn.com/AnOrRt
http://on-msn.com/y9tcUL
http://on-msn.com/yz0CFJ
http://on-msn.com/xoGfhV
http://on-msn.com/znULGL
http://on-msn.com/xTewH4
http://on-msn.com/AEJQMM
http://on-msn.com/zacSrr
http://on-msn.com/wneSnL
http://on-msn.com/Awoa8O
http://on-msn.com/x5jdBN
http://on-msn.com/AnMuDN
http://on-msn.com/Av0ZzG
http://on-msn.com/yvbd7

http://on-msn.com/xkXkdZ
http://on-msn.com/zZVJgW
http://on-msn.com/zKpPQ9
http://on-msn.com/xV1UOM
http://on-msn.com/zbTwFA
http://on-msn.com/zqhPfL
http://on-msn.com/z4QdFd
http://on-msn.com/z0RX72
http://on-msn.com/xzvy9p
http://on-msn.com/wik8wz
http://on-msn.com/y7FZhe
http://on-msn.com/ym0sPy
http://on-msn.com/zMXrdv
http://on-msn.com/Ai1RWB
http://on-msn.com/woGps8
http://on-msn.com/xAaz7z
http://on-msn.com/yCiK6v
http://on-msn.com/AxjxXp
http://on-msn.com/xvo9yc
http://on-msn.com/yGRlrX
http://on-msn.com/yWbnyF
http://on-msn.com/x6dIs7
http://on-msn.com/yZFc4N
http://on-msn.com/Ahv48H
http://on-msn.com/yJFfNw
http://on-msn.com/wnbnmd
http://on-msn.com/wl2eg2
http://on-msn.com/y51RFF
http://on-msn.com/wh4anc
http://on-msn.com/AC3W6A
http://on-msn.com/zZX9g8
http://on-msn.com/w4VXSg
http://on-msn.com/AtgD6t
http://on-msn.com/AADxMY
http://on-msn.com/wSMB0u
http://on-msn.com/zGYHsx
http://on-msn.com/Aw5Qmr
http://on-msn.com/Ao52Ss
http://on-msn.com/y5Dd3T
http://on-msn.com/A3G8id


In [ ]:
# Scrapt form web
df_temp = pd.DataFrame()
list_url = frame['url'].values

In [ ]:
frame.iloc[:,2][0]

In [ ]:
test_url= 'http://on-msn.com/O5Y3Pe'

res = requests.get(test_url)
if res.status_code  == 200:
#     soup = BeautifulSoup(res.content, 'lxml')
#     author = soup.find("span",{"class":"byline"})
#     print(author)
#     content = soup.find("div", {"class": "article-body"})
#     print(content)

    # Initialize XPath
    xpath_selector = Selector(text=res.text)
    
    author = xpath_selector.xpath('//*[@id="post-22962"]/header/div[2]/p/span/a/strong/text()').extract()[0]
    print("Author: ", author)
    print("\n")
    content = xpath_selector.xpath('//div[@class=" card-content "]/p/text()').extract()
    content = join_txt(content)
    print(content)
else:
    print("[ERROR]: ", res.status_code)

In [ ]:
# -------------------------------------------------
frame.head()

In [ ]:
# -------------------------------------------------
frame.shape